<a href="https://colab.research.google.com/github/kridtapon/WFO-Future-Trend-Crossover/blob/main/WFO_Future_Trend_Crossover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.3 MB/s eta 0:00:00


In [9]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate Displaced Moving Averages (DMA)
def calculate_dma(df, period, shift):
    return df['Close'].rolling(window=period).mean().shift(shift)

# Define the stock symbol and time period
symbol = 'HWM' # HWM AVGO NVDA WMT
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Walk-forward optimization with DMA

def walk_forward_optimization_dma(df, start_year, end_year):
    results = []

    # Define dynamic ranges for DMA parameters
    dma_fast_period_range = range(1, 51, 5)
    dma_slow_period_range = range(51, 101, 5)
    dma_fast_shift_range = range(1, 21)
    dma_slow_shift_range = range(1, 21)

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Loop through all combinations of DMA parameters
        for params in itertools.product(dma_fast_period_range, dma_slow_period_range, dma_fast_shift_range, dma_slow_shift_range):
            fast_period, slow_period, fast_shift, slow_shift = params

            # Calculate DMAs on the training data
            train_data['DMA_fast'] = calculate_dma(train_data, fast_period, fast_shift)
            train_data['DMA_slow'] = calculate_dma(train_data, slow_period, slow_shift)

            # Generate entry and exit signals
            entries = train_data['DMA_fast'] > train_data['DMA_slow']
            exits = train_data['DMA_fast'] < train_data['DMA_slow']

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = params

        # Test with the best parameters on the test data
        test_data['DMA_fast'] = calculate_dma(test_data, best_params[0], best_params[2])
        test_data['DMA_slow'] = calculate_dma(test_data, best_params[1], best_params[3])

        entries = test_data['DMA_fast'] > test_data['DMA_slow']
        exits = test_data['DMA_fast'] < test_data['DMA_slow']

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)

# Perform walk-forward optimization
results = walk_forward_optimization_dma(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    # Apply optimized DMA indicators
    yearly_data['DMA_fast'] = calculate_dma(yearly_data, params[0], params[2])
    yearly_data['DMA_slow'] = calculate_dma(yearly_data, params[1], params[3])

    # Define entry/exit conditions
    entries = yearly_data['DMA_fast'] > yearly_data['DMA_slow']
    exits = yearly_data['DMA_fast'] < yearly_data['DMA_slow']

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using the combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-9-5a46661f53b7>:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-9-5a46661f53b7>:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-9-5a46661f53b7>:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in


Walk-Forward Optimization Results:
   Year       Best_Params
0  2020     (1, 51, 4, 2)
1  2021  (36, 51, 12, 12)
2  2022  (36, 51, 12, 12)
3  2023  (36, 51, 13, 13)
4  2024   (11, 51, 13, 5)
5  2025   (26, 61, 17, 3)
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           322698.663118
Total Return [%]                       222.698663
Benchmark Return [%]                   250.750162
Max Gross Exposure [%]                      100.0
Total Fees Paid                       5545.425576
Max Drawdown [%]                        26.790754
Max Drawdown Duration                       501.0
Total Trades                                   15
Total Closed Trades                            14
Total Open Trades                               1
Open Trade PnL                      -23389.316415
Win Rate [%]                    

<ipython-input-9-5a46661f53b7>:108: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-9-5a46661f53b7>:109: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-9-5a46661f53b7>:108: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-9-5a466

In [10]:
# Filter Test Years
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Buy and Hold Performance Metrics
df_holding = df['Close']
pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,350750.161547
Total Return [%],250.750162
Benchmark Return [%],250.750162
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],68.916937
